In [1]:
import pandas as pd
import json

from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from main import query_graph, refine_query, query_graph_cot
from tqdm import tqdm
tqdm.pandas()

2024-11-08 21:54:04.458 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-11-08 21:54:04.505 
  command:

    streamlit run C:\Users\SLY\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
df = pd.read_json('datasets/HotPotQA/hotpot_dev_fullwiki_v1.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               7405 non-null   object
 1   answer            7405 non-null   object
 2   question          7405 non-null   object
 3   supporting_facts  7405 non-null   object
 4   context           7405 non-null   object
 5   type              7405 non-null   object
 6   level             7405 non-null   object
dtypes: object(7)
memory usage: 405.1+ KB


## Experiment


To run the evaluation script

```
python hotpot_evaluate_v1.py <path_to_prediction> <path_to_gold>
```

Example:

```
python hotpot_evaluate_v1.py <sample_dev_pred.json> <hotpot_dev_fullwiki_v1.json>
```

In [3]:
context_list = df['context']

In [4]:
df.head()

,_id,answer,question,supporting_facts,context,type,level
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]","[[Adam Collis, [Adam Collis is an American fil...",comparison,hard
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...","[[A Kiss for Corliss, [A Kiss for Corliss is a...",bridge,hard
2,5a85ea095542994775f606a8,Animorphs,"What science fantasy young adult series, told ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj...","[[Animorphs, [Animorphs is a science fantasy s...",bridge,hard
3,5adbf0a255429947ff17385a,no,Are the Laleli Mosque and Esma Sultan Mansion ...,"[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]","[[Esma Sultan, [Esma Sultan is the name of thr...",comparison,hard
4,5a8e3ea95542995a26add48d,"Greenwich Village, New York City","The director of the romantic comedy ""Big Stone...","[[Big Stone Gap (film), 0], [Adriana Trigiani,...","[[Great Eastern Conventions, [Great Eastern Co...",bridge,hard


In [5]:
len(context_list[3:100])

97

In [6]:
# propositions = []

# for i, value in enumerate(tqdm(context_list[3:100])):
#         for j, context in enumerate(value):
#                 index = 1
#                 if index < len(context):
#                     words = context[1]
#                     combined = " ".join(words)
#                     get_propositions_nosplit(combined, propositions)

In [7]:
# # Open the file in read mode
# with open('propositions.txt', 'r') as file:
#     # Read all lines and store them in a list
#     propositions_from_file = [line.strip() for line in file]

# print(propositions_from_file)

In [8]:
# list_of_edges = generateEdges(propositions_from_file)

In [9]:
# if createGraph(list_of_edges):
#     print("Success")
# else:
#     print("Failed")

In [10]:
# def find_supporting_facts(question, answer, context):
#     supporting_facts = []
#     for passage in context:
#         title, sentences = passage
#         for i, sentence in enumerate(sentences):
#             if answer in sentence or any(word in sentence for word in question.split()):
#                 supporting_facts.append([title, i])
#     return supporting_facts

In [11]:
# Define a function to apply to each row
def print_qa(row):
    
    question = row['question']
    questionid = row['_id']
    result = query_graph_cot(question)

    if result is not None:
    
        model_answer = result['result']

        if "don't know the answer" in model_answer:
            model_answer = ""
    
        if len(row['answer']) > 0:
            real_answer = row['answer']
        else:
            real_answer = "\"\""

        if len(result['intermediate_steps']) > 0:
            cypher_query = result['intermediate_steps'][0]['query']
            context = result['intermediate_steps'][1]['context']
       
            if(len(context) < 1):
            #     result = refine_query(cypher_query[6:], question)
            #     model_answer = result['result'] 
                model_answer = "\"\""
                my_dict.update({questionid: ""})
            else:
                my_dict.update({questionid: model_answer})
                
        with open("preds.txt", "a") as preds:
            preds.write("question: " + question + "\n")
            preds.write("real_answer: " + real_answer + "\n")
            preds.write("model_answer: " + model_answer + "\n")  
            preds.write("cypher_query: " + cypher_query + "\n")
            preds.write("======================" + "\n")  
        preds.close()

    else: 
        my_dict.update({questionid: ""})
        

In [12]:
df[0:2]

,_id,answer,question,supporting_facts,context,type,level
0,5a8b57f25542995d1e6f1371,yes,Were Scott Derrickson and Ed Wood of the same ...,"[[Scott Derrickson, 0], [Ed Wood, 0]]","[[Adam Collis, [Adam Collis is an American fil...",comparison,hard
1,5a8c7595554299585d9e36b6,Chief of Protocol,What government position was held by the woman...,"[[Kiss and Tell (1945 film), 0], [Shirley Temp...","[[A Kiss for Corliss, [A Kiss for Corliss is a...",bridge,hard


In [13]:
my_dict = {}
# Apply the function to each row
df[0:2].progress_apply(print_qa, axis=1)
    
with open("dev_fullwiki_pred.json", "a") as json_file:
    json.dump(my_dict, json_file, indent=4)

100%|██████████| 2/2 [00:00<?, ?it/s]

input_variables=['question', 'schema'] template="\nYou are an expert Neo4j Cypher translator who converts English to Cypher based on the Neo4j Schema provided, following the instructions below:\n\nschema: {schema}\n\nQuestion: {question}\n\nLet's break down this question step-by-step to construct the correct Cypher query:\n\n1. **Identify Entities**: Determine the main entities (e.g., person, place, item) that need to be included in the query.\n2. **Identify Relationships**: Identify the relationships between entities, as described in the question.\n3. **Define Filters and Conditions**: Determine any specific conditions, properties, or filters required (e.g., name, age, location).\n4. **Determine the Output**: Decide what information should be returned (e.g., node properties, relationships).\n5. **Construct the Query**: Using the information above, write the Cypher query.\n\nCypher Query:\n"
input_variables=['question', 'schema'] template="\nYou are an expert Neo4j Cypher translator wh